#WE LOAD THE DATASET HERE AND #CHECK IT FOR VARIOUS PARAMETERS## EDA AND THEN CLEANING

In [5]:
# === Cell 1: Setup (run first) ===============================================
import os, gc, textwrap, numpy as np, pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
print("✅ Libraries loaded.")


✅ Libraries loaded.


In [12]:
# === Cell 2 (Option A): Load from local upload ================================
# If your CSV is on your computer, run this cell, pick the file, and use that path below.
from google.colab import files
#uploaded = files.upload()
#csv_path = pd.read_csv("/content/Fraud Detection System for JPMorgan Chase _log.csv")#list(uploaded.keys())[0]

csv_path = "/content/Fraud Detection System for JPMorgan Chase _log.csv"
print("📄 Using file:", csv_path)

📄 Using file: /content/Fraud Detection System for JPMorgan Chase _log.csv


In [13]:
# === Cell 2 (Option B): Load from Google Drive ===============================
# If your CSV is in Drive, run this instead of Cell 2(A), and set csv_path accordingly.
# from google.colab import drive
# drive.mount('/content/drive')
# csv_path = "/content/drive/MyDrive/path/to/your/Fraud Detection System for JPMorgan Chase _log.csv"
# print("📄 Using file:", csv_path)


In [14]:
# === Cell 3: Memory-optimized read ===========================================
# Known columns in this dataset
usecols = [
    "step","type","amount","nameOrig","oldbalanceOrg","newbalanceOrig",
    "nameDest","oldbalanceDest","newbalanceDest","isFraud","isFlaggedFraud"
]

dtypes = {
    "step": "int32",
    "type": "category",
    "amount": "float32",
    "nameOrig": "category",
    "oldbalanceOrg": "float32",
    "newbalanceOrig": "float32",
    "nameDest": "category",
    "oldbalanceDest": "float32",
    "newbalanceDest": "float32",
    "isFraud": "int8",
    "isFlaggedFraud": "int8",
}

df = pd.read_csv(csv_path, usecols=usecols, dtype=dtypes)
print("✅ Loaded shape:", df.shape)
print(df.dtypes)


✅ Loaded shape: (6362620, 11)
step                 int32
type              category
amount             float32
nameOrig          category
oldbalanceOrg      float32
newbalanceOrig     float32
nameDest          category
oldbalanceDest     float32
newbalanceDest     float32
isFraud               int8
isFlaggedFraud        int8
dtype: object


In [15]:
# === Cell 4: Basic sanity checks =============================================
print("Missing values per column:\n", df.isna().sum())
print("\nTransaction types:\n", df["type"].value_counts())
fraud_rate = df["isFraud"].mean()
print(f"\nFraud rate: {fraud_rate:.4%}")

# Quick peek
df.head(3)


Missing values per column:
 step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Transaction types:
 type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

Fraud rate: 0.1291%


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,"9,839.6396",C1231006815,"170,136.0000","160,296.3594",M1979787155,0.0000,0.0000,0,0
1,1,PAYMENT,"1,864.2800",C1666544295,"21,249.0000","19,384.7207",M2044282225,0.0000,0.0000,0,0
2,1,TRANSFER,181.0000,C1305486145,181.0000,0.0000,C553264065,0.0000,0.0000,1,0


In [16]:
# === Cell 5: Create safe numeric helpers =====================================
EPS = 1e-6

# Origin/destination "consistency" deltas (useful for anomaly signals)
# origin_delta ~ oldbalanceOrg - amount - newbalanceOrig
# dest_delta   ~ newbalanceDest - oldbalanceDest - amount
df["origin_delta"] = (df["oldbalanceOrg"] - df["amount"] - df["newbalanceOrig"]).astype("float32")
df["dest_delta"]   = (df["newbalanceDest"] - df["oldbalanceDest"] - df["amount"]).astype("float32")

# Ratios / flags that often help
df["amt_to_oldOrg_ratio"] = (df["amount"] / (df["oldbalanceOrg"].abs() + EPS)).astype("float32")
df["is_zero_bal_sender"]  = (df["oldbalanceOrg"].abs() < EPS).astype("int8")
df["is_zero_bal_dest"]    = (df["oldbalanceDest"].abs() < EPS).astype("int8")

# Absolute error flags (tune thresholds later during modeling)
df["origin_mismatch_flag"] = (df["origin_delta"].abs() > 1.0).astype("int8")
df["dest_mismatch_flag"]   = (df["dest_delta"].abs() > 1.0).astype("int8")

print("✅ Feature columns added.")
df.head(3)


✅ Feature columns added.


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,origin_delta,dest_delta,amt_to_oldOrg_ratio,is_zero_bal_sender,is_zero_bal_dest,origin_mismatch_flag,dest_mismatch_flag
0,1,PAYMENT,"9,839.6396",C1231006815,"170,136.0000","160,296.3594",M1979787155,0.0000,0.0000,0,0,0.0000,"-9,839.6396",0.0578,0,1,0,1
1,1,PAYMENT,"1,864.2800",C1666544295,"21,249.0000","19,384.7207",M2044282225,0.0000,0.0000,0,0,0.0000,"-1,864.2800",0.0877,0,1,0,1
2,1,TRANSFER,181.0000,C1305486145,181.0000,0.0000,C553264065,0.0000,0.0000,1,0,0.0000,-181.0000,1.0000,0,1,0,1


from matplotlib import pyplot as plt
_df_57['amount'].plot(kind='hist', bins=20, title='amount')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_58['oldbalanceOrg'].plot(kind='hist', bins=20, title='oldbalanceOrg')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_59['newbalanceOrig'].plot(kind='hist', bins=20, title='newbalanceOrig')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['step']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'step'}, axis=1)
              .sort_values('step', ascending=True))
  xs = counted['step']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_68.sort_values('step', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('step')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_69['amount'].plot(kind='line', figsize=(8, 4), title='amount')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_70['oldbalanceOrg'].plot(kind='line', figsize=(8, 4), title='oldbalanceOrg')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_71['newbalanceOrig'].plot(kind='line', figsize=(8, 4), title='newbalanceOrig')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_72['isFraud'].plot(kind='line', figsize=(8, 4), title='isFraud')
plt.gca().spines[['top', 'right']].set_visible(False)

In [17]:
# === Cell 6: Optional — derive a datetime from 'step' ========================
# 'step' is typically an hour index in this dataset. If you want a concrete timestamp, set a start date.
# This is optional and mainly for EDA convenience.
START_TS = np.datetime64("2024-01-01T00:00:00")
df["txn_time"] = START_TS + pd.to_timedelta(df["step"].astype(int), unit="h")
df["txn_time"] = df["txn_time"].astype("datetime64[ns]")
print("🕒 txn_time created from step (assumes step=hour index).")
df[["step","txn_time"]].head(3)


🕒 txn_time created from step (assumes step=hour index).


,step,txn_time
0,1,2024-01-01 01:00:00
1,1,2024-01-01 01:00:00
2,1,2024-01-01 01:00:00


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['step']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'step'}, axis=1)
              .sort_values('step', ascending=True))
  xs = counted['step']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_50.sort_values('step', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('step')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['txn_time']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'txn_time'}, axis=1)
              .sort_values('txn_time', ascending=True))
  xs = counted['txn_time']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_51.sort_values('txn_time', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('txn_time')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_52['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_53.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['step']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'step'}, axis=1)
              .sort_values('step', ascending=True))
  xs = counted['step']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_54.sort_values('step', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('step')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['txn_time']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'txn_time'}, axis=1)
              .sort_values('txn_time', ascending=True))
  xs = counted['txn_time']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_55.sort_values('txn_time', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('txn_time')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_56['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

In [18]:
# === Cell 6: Optional — derive a datetime from 'step' ========================
# 'step' is typically an hour index in this dataset. If you want a concrete timestamp, set a start date.
# This is optional and mainly for EDA convenience.
START_TS = np.datetime64("2024-01-01T00:00:00")
df["txn_time"] = START_TS + pd.to_timedelta(df["step"].astype(int), unit="h")
df["txn_time"] = df["txn_time"].astype("datetime64[ns]")
print("🕒 txn_time created from step (assumes step=hour index).")
df[["step","txn_time"]].head(3)


🕒 txn_time created from step (assumes step=hour index).


,step,txn_time
0,1,2024-01-01 01:00:00
1,1,2024-01-01 01:00:00
2,1,2024-01-01 01:00:00


In [19]:
# === Cell 7: Light integrity checks ==========================================
summary = {
    "rows": len(df),
    "cols": df.shape[1],
    "n_customers": df["nameOrig"].nunique(),
    "n_destinations": df["nameDest"].nunique(),
    "fraud_rate": float(df["isFraud"].mean()),
}
summary


{'rows': 6362620,
 'cols': 19,
 'n_customers': 6353307,
 'n_destinations': 2722362,
 'fraud_rate': 0.001290820448180152}

In [20]:
# === Cell 8: Save processed dataset ==========================================
# Save a full processed Parquet (fast & compressed)
os.makedirs("processed", exist_ok=True)
full_out = "processed/fraud_processed.parquet"
df.to_parquet(full_out, index=False)
print("💾 Saved:", full_out)

# Also save a manageable sample for fast EDA/model prototyping (e.g., 200k rows)
np.random.seed(42)
sample_n = min(200_000, len(df))
df_sample = df.sample(sample_n)
sample_out = "processed/fraud_processed_sample_200k.parquet"
df_sample.to_parquet(sample_out, index=False)
print("💾 Saved sample:", sample_out)

# Free some memory if needed
del df_sample; gc.collect()


💾 Saved: processed/fraud_processed.parquet
💾 Saved sample: processed/fraud_processed_sample_200k.parquet


52339

In [21]:
# === Cell 9: Quick report (so you can paste into your doc) ===================
report = f"""
Rows: {summary['rows']:,}
Columns: {summary['cols']}
Unique senders: {summary['n_customers']:,}
Unique destinations: {summary['n_destinations']:,}
Fraud rate: {summary['fraud_rate']:.4%}

New features added:
- origin_delta, dest_delta
- amt_to_oldOrg_ratio
- is_zero_bal_sender, is_zero_bal_dest
- origin_mismatch_flag, dest_mismatch_flag
- txn_time (derived from step, optional)
"""
print(textwrap.dedent(report))



Rows: 6,362,620
Columns: 19
Unique senders: 6,353,307
Unique destinations: 2,722,362
Fraud rate: 0.1291%

New features added:
- origin_delta, dest_delta
- amt_to_oldOrg_ratio
- is_zero_bal_sender, is_zero_bal_dest
- origin_mismatch_flag, dest_mismatch_flag
- txn_time (derived from step, optional)



Here’s a clear, submission-ready bullet point list with explanations for your **Data Ingestion & Preprocessing** step.
You can put this into your capstone document under *Methodology → Step 1*.

---

## **Data Ingestion & Preprocessing — Steps & Explanations**

1. **Imported Required Libraries**

   * Used `pandas` for data handling, `numpy` for numerical operations, and system utilities for file management.
   * Set Pandas display options for better visibility during analysis.

2. **Loaded the Dataset into the Environment**

   * Uploaded the provided CSV file directly into Google Colab.
   * Defined `usecols` to load only relevant columns, and set **data types (`dtype`)** to memory-efficient formats like `int8`, `float32`, and `category`.
   * This reduces memory usage and speeds up processing for large datasets (\~6.3 million rows).

3. **Initial Data Inspection**

   * Checked dataset shape, column names, and data types.
   * Verified transaction types and calculated the overall fraud rate.
   * Counted missing values to confirm data completeness.

4. **Feature Creation for Fraud Analysis**

   * **`origin_delta`** = Difference between expected and actual sender balances after the transaction.
   * **`dest_delta`** = Difference between expected and actual receiver balances after the transaction.
   * **`amt_to_oldOrg_ratio`** = Ratio of transaction amount to sender’s previous balance.
   * **`is_zero_bal_sender` & `is_zero_bal_dest`** = Flags indicating zero balance before transaction (potentially suspicious).
   * **`origin_mismatch_flag` & `dest_mismatch_flag`** = Flags for mismatches greater than a threshold in balance changes.
   * These features can help models identify anomalies linked to fraudulent activity.

5. **Optional Time Feature Engineering**

   * Converted `step` (hour index) into an actual timestamp (`txn_time`) for time-based EDA.
   * Assumed a start date (e.g., `2024-01-01`) and added hours accordingly.

6. **Data Integrity Checks**

   * Counted unique senders and destinations to understand dataset diversity.
   * Verified that fraud labels (`isFraud`) were correctly loaded.

7. **Data Saving for Future Steps**

   * Saved **full processed dataset** as a Parquet file for efficient loading in modeling stages.
   * Created and saved a **200k row sample** for quick prototyping and EDA.